<a href="https://colab.research.google.com/github/shivar2/MotorImagery-Classification/blob/main/Preproccess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install moabb & braindecode

In [1]:
pip install moabb

In [2]:
pip install braindecode

In [3]:
pip install -U https://api.github.com/repos/shivar2/braindecode/zipball/master

  Using cached https://api.github.com/repos/shivar2/braindecode/zipball/master
  Created wheel for Braindecode: filename=Braindecode-0.5-cp37-none-any.whl size=67286 sha256=7618cac467ef694fd35e705d54c214fbfa44beddf7bee361ee192f4a09cf230c
  Stored in directory: /tmp/pip-ephem-wheel-cache-epkb9iib/wheels/e8/96/7e/9b71798fc04d312683ade7e3a639b36579b720176c6a54dc86
Successfully built Braindecode
  Found existing installation: Braindecode 0.5
    Uninstalling Braindecode-0.5:
      Successfully uninstalled Braindecode-0.5


## connect to drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os, sys
os.chdir('/content/drive/MyDrive/MotorImagery-Classification')

# preproccess

**load data**


In [6]:
from braindecode.datasets.moabb import MOABBDataset
subject_id = 1
dataset = MOABBDataset(dataset_name="Schirrmeister2017", subject_ids=[subject_id], path="dataset")

# subject_id = 1
# dataset = MOABBDataset(dataset_name="BNCI2014001", subject_ids=[subject_id])


Creating RawArray with float64 data, n_channels=128, n_times=1225545
    Range : 0 ... 1225544 =      0.000 ...  2451.088 secs
Ready.
Creating RawArray with float64 data, n_channels=128, n_times=616535
    Range : 0 ... 616534 =      0.000 ...  1233.068 secs
Ready.
320 events found
Event IDs: [1 2 3 4]
160 events found
Event IDs: [1 2 3 4]


filter

In [7]:
from braindecode.datautil.preprocess import (
    exponential_moving_standardize, preprocess, Preprocessor)

low_cut_hz = 4.  # low cut frequency for filtering
high_cut_hz = 38.  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda x: x * 1e6),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    # Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
    #              factor_new=factor_new, init_block_size=init_block_size)
]

# Transform the data
preprocess(dataset, preprocessors)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 38.00 Hz
- Upper transition bandwidth: 9.50 Hz (-6 dB cutoff frequency: 42.75 Hz)
- Filter length: 825 samples (1.650 sec)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 38 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 

## Cut Compute Windows

In [11]:
import numpy as np
from braindecode.datautil.windowers import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)


Used Annotations descriptions: ['feet', 'left_hand', 'rest', 'right_hand']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
320 matching events found
No baseline correction applied
0 projection items activated
Loading data for 320 events and 2250 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['feet', 'left_hand', 'rest', 'right_hand']
Adding metadata with 4 columns
Replacing existing metadata with 4 columns
160 matching events found
No baseline correction applied
0 projection items activated
Loading data for 160 events and 2250 original time points ...
0 bad epochs dropped


## Split dataset into train and valid

In [12]:
splitted = windows_dataset.split('run')
train_set = splitted['train']
valid_set = splitted['test']

## save sets

In [15]:
import torch

torch.save(train_set, 'data-file/Schirrmeister2017/train/' + subject_id + '.pt')
torch.save(valid_set, 'data-file/Schirrmeister2017/test/' + str(subject_id) + '.pt')